In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
data = []

pedago_period = [249108, 942175]
semesters = ['semester 1', 'semester 6']
acad_period = [978181, 978187, 978195, 39486325, 123455150, 123456101, 213637754, 213637922, 213638028, 355925344]
years = ['2007-2008','2008-2009','2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017']

for pedago in pedago_period:
    for acad in acad_period:
        payload = {'ww_x_GPS': '-1',
                   'ww_i_reportModel': 133685247,
                   'ww_i_reportModelXsl': 133685270,
                   'ww_x_UNITE_ACAD': 249847,
                   'ww_x_PERIODE_ACAD': acad,
                   'ww_x_PERIODE_PEDAGO': pedago,
                   'ww_x_HIVERETE': 'null'
                  }
        data.append(requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html', params=payload))

In [6]:
soups = []
for i in range(0, 20):
    soups.append(BeautifulSoup(data[i].text, 'html.parser'))


In [36]:
from math import floor

students = []
i = 0
for fsoup in soups:
    for tr in fsoup.html.body.hr.table.findAll('tr'):
        tds = tr.findAll('td')
        if (len(tds) > 0):
            student = {'civilite': tds[0].text,
                       'name': tds[1].text,
                       'sciper': tds[10].text,
                       'year': years[i%10],
                       'semester': semesters[floor(i/10)]
                      }
            students.append(student)
    i += 1
    if (i == 10):
        students_sem1 = pd.DataFrame(students)
        students = []
    elif (i == 20):
        students_sem6 = pd.DataFrame(students)

students_sem6.drop('civilite', axis=1, inplace=True)
students_sem6.drop('name', axis=1, inplace=True)

students_sem1.drop_duplicates(subset=['sciper'], keep='first', inplace=True)
students_sem1.set_index(['sciper'], inplace=True)

students_sem6.drop_duplicates(subset=['sciper'], keep='last', inplace=True)
students_sem6.set_index(['sciper'], inplace=True)

students = pd.concat([students_sem1, students_sem6], axis=1, join='inner')
students.drop('semester', axis=1, inplace=True)
students.columns = ['civilite', 'name', 'arrival year', 'graduation year']

students['duration'] = [int(dep.split("-")[1]) - int(arr.split("-")[0]) for arr, dep in students[['arrival year', 'graduation year']].values]
students['duration'] *= 10

students.head()

,civilite,name,arrival year,graduation year,duration
sciper,,,,,
169569,Monsieur,Arévalo Christian,2007-2008,2009-2010,30
179449,Monsieur,Bindschaedler Vincent,2007-2008,2009-2010,30
178553,Monsieur,Bloch Marc-Olivier,2007-2008,2009-2010,30
178271,Monsieur,Boéchat Marc-Alexandre,2007-2008,2009-2010,30
180731,Monsieur,Bricola Jean-Charles,2007-2008,2009-2010,30
